In [2]:
import numpy as np
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

In [3]:
# Function to compute the Lennard-Jones potential and force
def lj_potential_and_force(r, sigma, epsilon):
    sr6 = (sigma / r) ** 6
    sr12 = sr6 ** 2
    potential = 4 * epsilon * (sr12 - sr6)
    force = 24 * epsilon * (2 * sr12 - sr6) / r
    return potential, force

In [4]:
# Function to compute the total potential energy and forces
def compute_forces(positions, num_particles, box_length, sigma, epsilon, r_cut):
    forces = np.zeros_like(positions)
    potential_energy = 0.0
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            # Minimum image convention
            rij = positions[i] - positions[j]
            rij = rij - box_length * np.rint(rij / box_length)
            r2 = np.dot(rij, rij)
            if r2 < r_cut ** 2:
                r = np.sqrt(r2)
                potential, force = lj_potential_and_force(r, sigma, epsilon)
                potential_energy += potential
                force_vector = force * rij / r
                forces[i] += force_vector
                forces[j] -= force_vector
    return potential_energy, forces

In [13]:
# Function to read XYZ file
def read_xyz(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    num_particles = int(lines[0].strip())
    positions = []
    for line in lines[2:2+num_particles]:
        _, x, y, z = line.split()
        positions.append([float(x), float(y), float(z)])
    
    positions = np.array(positions)
    return positions, num_particles

In [14]:
# Function to write XYZ file
def write_xyz(file_path, positions, step):
    with open(file_path, 'a') as f:
        f.write(f"{len(positions)}\n")
        f.write(f"Step {step}\n")
        for pos in positions:
            f.write(f"Ar {pos[0]} {pos[1]} {pos[2]}\n")

In [7]:
# Function to write energy data
def write_energy(file_path, step, potential_energy, kinetic_energy, total_energy):
    with open(file_path, 'a') as f:
        f.write(f"Step {step}, Potential Energy: {potential_energy:.4f}, Kinetic Energy: {kinetic_energy:.4f}, Total Energy: {total_energy:.4f}\n")

In [8]:
# Velocity Verlet integration
def velocity_verlet(positions, velocities, forces, num_particles, box_length, time_step, sigma, epsilon, r_cut):
    positions += velocities * time_step + 0.5 * forces * time_step ** 2
    positions = positions % box_length  # Apply periodic boundary conditions
    potential_energy, new_forces = compute_forces(positions, num_particles, box_length, sigma, epsilon, r_cut)
    velocities += 0.5 * (forces + new_forces) * time_step
    return positions, velocities, new_forces, potential_energy

In [9]:
#Parameter Initialization
sigma = 3.4  # in Angstroms
epsilon = 0.238  # in kcal/mol

# Simulation parameters
temperature = 100.0  # Temperature in Kelvin
r_cut = 2.5 * sigma  # Cutoff distance, scaled by sigma
kB = 1.9872041e-3  # Boltzmann constant in kcal/(mol*K)

# MD simulation parameters
num_particles = 100
box_length = 10.0  # Angstroms
time_step = 0.001  # Picoseconds
num_steps = 100  # Total number of steps
temperature = 120.0  # Kelvin

In [15]:
#Initialize velocities
positions, num_particles= read_xyz('argons.xyz')
velocities = np.random.randn(num_particles, 3)
velocities -= np.mean(velocities, axis=0)  # Zero total momentum
velocities *= np.sqrt(kB * temperature / epsilon)  # Scale velocities to match the desired temperature

In [16]:
# Output files
xyz_output_file = 'md_trajectory.xyz'
energy_output_file = 'md_energy.txt'

# Clear the output files if they already exist
open(xyz_output_file, 'w').close()
open(energy_output_file, 'w').close()

In [17]:
# Run the MD simulation
forces = np.zeros_like(positions)
potential_energy, forces = compute_forces(positions, num_particles, box_length, sigma, epsilon, r_cut)

for step in range(num_steps):
    positions, velocities, forces, potential_energy = velocity_verlet(
        positions, velocities, forces, num_particles, box_length, time_step, sigma, epsilon, r_cut)
    
    kinetic_energy = 0.5 * np.sum(velocities ** 2) * (epsilon / (kB * temperature))
    total_energy = potential_energy + kinetic_energy
    
    # Write the positions to the XYZ file
    write_xyz(xyz_output_file, positions, step)
    
    # Write the energy data to the text file
    write_energy(energy_output_file, step, potential_energy, kinetic_energy, total_energy)
    
    if step % 1 == 0:
        print(f"Step {step}/{num_steps}, Total Energy: {total_energy:.4f}")

Step 0/100, Total Energy: 43108.1942
Step 1/100, Total Energy: 45100.6800
Step 2/100, Total Energy: 45493.1230
Step 3/100, Total Energy: 45556.6797
Step 4/100, Total Energy: 45580.3956
Step 5/100, Total Energy: 45603.5487
Step 6/100, Total Energy: 45610.4358
Step 7/100, Total Energy: 45596.9886
Step 8/100, Total Energy: 45581.9245
Step 9/100, Total Energy: 45575.1628
Step 10/100, Total Energy: 45576.7030
Step 11/100, Total Energy: 45582.2446
Step 12/100, Total Energy: 45585.7123
Step 13/100, Total Energy: 45584.4223
Step 14/100, Total Energy: 45580.4927
Step 15/100, Total Energy: 45575.8063
Step 16/100, Total Energy: 45567.5959
Step 17/100, Total Energy: 45546.1500
Step 18/100, Total Energy: 45496.2612
Step 19/100, Total Energy: 45463.2439
Step 20/100, Total Energy: 45702.0950
Step 21/100, Total Energy: 45694.4124
Step 22/100, Total Energy: 45459.9704
Step 23/100, Total Energy: 45493.5917
Step 24/100, Total Energy: 45541.0380
Step 25/100, Total Energy: 45560.3275
Step 26/100, Total Ene

In [19]:
import mdtraj as md
import nglview as nv

# Define the atom types as needed
num_atoms = 35  # Replace with the actual number of atoms in your XYZ file
atom_types = ['Ar'] * num_atoms  # Adjust this line if your atoms are different

# Create a simple topology for the XYZ file
top = md.Topology()
chain = top.add_chain()
residue = top.add_residue("RES", chain)
for atom_type in atom_types:
    top.add_atom(atom_type, md.element.Element.getBySymbol(atom_type), residue)

# Load the XYZ file with the created topology
traj = md.load_xyz('md_trajectory.xyz', top=top)
# Visualize the trajectory using nglview
view = nv.show_mdtraj(traj)
view.clear_representations()
view.add_spacefill(radiusType='radius', radius=0.5)
view

NGLWidget(max_frame=99)